토픽 모델링

비정형 텍스트 뭉치에서 주제(토픽)들을 추출해주는 확률모델 알고리즘이다.
주로 LDA( Latent Dirichlet Allocation )또는 그 변용 기법들을 사용한다.
문학들의 주제분석, 주제 분포 변화에 따른 학문분야의 연구동향 파악 등의 연구에 적용됐다.

쉽게 말하면 "LDA를 이용해 문서군의 주제를 얻어내는 기술"이겠네요. 그런데 여러분들이 착각하지 말아야 될 것이 있습니다. 보통 주제라고 하면 '정치','경제'처럼 추상화된 한 단어를 생각하게 되는데요. 토픽 모델링에서 말하는 토픽이란, 문서로부터 추출된 단어들의 집합을 의미합니다. 예를 들어 보겠습니다.



문서 A	

국토연구원이 최근 발표한 ‘생애주기별 주거소비 특성을 반영한 주거지원 방안’에 따르면 20~30대가 주택금융을 이용할 때 자산 제약에 직면할 가능성은 21%로 가장 높게 나타났다. 

문서 B	

국회 추가경정예산 처리 과정에서 당론과 달리 찬성표를 던진 자유한국당 장제원 의원이 이번에는 강제 당론 폐지를 주장했다.



A는 신문 기사의 경제면, B는 정치면에서 발췌한 내용입니다. 이 문서들로 토픽 모델링을 실행하면 결과는 아래와 같을 것입니다 : 



토픽1 - { '주거소비', '주택금융', '자산', .. }

토픽2 - { '국회', '자유한국당', '찬성표', '의원', .. }


이제 토픽과 토픽 모델링에 대해 감이 좀 오셨을거라 생각합니다. 주제라는건 추상적인 개념이기 때문에 에 컴퓨터가 알아서 떠올리는게 불가능하죠. 따라서, 다양한 문서들을 참고하여 구체적인 단어들 간의 연관성을 파악하고, 비슷한 분포를 갖는 단어들의 집합을 만들어 내는 것입니다. 그 집합이 그에 속한 단어들을 포괄하는 하나의 개념이 되는거라고 보는거죠. 그래서 토픽 모델링을 "개념 추출"이라고도 부릅니다.


nltk : 자연언어 처리를 위한 패키지

stop_word : 필요없는 단어 처리

gensim : LDA 모델이 포함하는 주제 모델링 패키지

    주요기능
    Topic modeling
    Latent Dirichlet allocation (LDA)
    Latent semantic indexing (LSI)
    Hierarchical Dirichlet process (HDP)
    Word embedding
    word2vec

In [92]:
!pip install -U nltk

Requirement already up-to-date: nltk in /Users/salab/anaconda3/lib/python3.6/site-packages
Requirement already up-to-date: six in /Users/salab/anaconda3/lib/python3.6/site-packages (from nltk)


In [93]:
!pip install gensim

In [94]:
!pip install stop_words

In [53]:
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from stop_words import get_stop_words
from gensim import corpora, models
import gensim
from nltk.tokenize import word_tokenize

그럼 파이썬으로 토픽 모델링을 해보겠습니다. 먼저 그럼 텍스트 뭉치가 있어야 겠죠 


In [76]:
doc_set = ["Brocolli is good to eat. My brother likes to eat good brocolli, but not my mother.",
           "My mother spends a lot of time driving my brother around to baseball practice.",
           "Some health experts suggest that driving may cause increased tension and blood pressure.",
           "I often feel pressure to perform well at school, but my mother never seems to drive my brother to do better.",
           "Health professionals say that brocolli is good for your health."]

In [82]:
stopwords = get_stop_words('en')

In [95]:
texts = []

단어 추출을 위해 토크나이징, 불용어 처리를 마친 corpus로 만들어 줍니다 :

In [96]:
for doc in doc_set:
    # -- 토크 나이징
    word_token = word_tokenize(doc.lower())

    # -- 불용어 제거
    stopped_tokens = [word for word in word_token if not word in stopwords]
    texts.append(stopped_tokens)


In [97]:
print(texts)

[['brocolli', 'good', 'eat', '.', 'brother', 'likes', 'eat', 'good', 'brocolli', ',', 'mother', '.'], ['mother', 'spends', 'lot', 'time', 'driving', 'brother', 'around', 'baseball', 'practice', '.'], ['health', 'experts', 'suggest', 'driving', 'may', 'cause', 'increased', 'tension', 'blood', 'pressure', '.'], ['often', 'feel', 'pressure', 'perform', 'well', 'school', ',', 'mother', 'never', 'seems', 'drive', 'brother', 'better', '.'], ['health', 'professionals', 'say', 'brocolli', 'good', 'health', '.']]


이 텍스트를 Gensim의 corpora.Dictionary 객체로 바꿔줍시다. 

Dictionary 객체는 LDA와 상호작용하기 위한 메소드를 가진 특별한 사전객체 입니다.

토큰화된 corpus를 파라미터로 넣어주면 자동으로 LDA를 위한 준비를 끝마친 사전객체가 됩니다 :


In [86]:
word_dictionary = corpora.Dictionary(texts)

In [87]:
print(word_dictionary)

Dictionary(35 unique tokens: [',', '.', 'brocolli', 'brother', 'eat']...)


Dictionary 객체는 자동으로 단어에 아이디를 할당한 두 가지 dictionary를 생성합니다. 하나는 token2id로, 해당 단어의 아이디에 접근할 수 있도록 매핑시킨 dictionary이고 다른 하나는 id2token으로, id로 단어에 접근할 수 있도록 매핑시킨 dictionary입니다 :



In [89]:
for idx, val in enumerate(word_dictionary): word_dictionary[idx]
print("\n[ word_dictionary.token2id ]")
print(word_dictionary.token2id)

print("\n[ word_dictionary.id2token ]")
print(word_dictionary.id2token)


[ word_dictionary.token2id ]
{',': 0, '.': 1, 'brocolli': 2, 'brother': 3, 'eat': 4, 'good': 5, 'likes': 6, 'mother': 7, 'around': 8, 'baseball': 9, 'driving': 10, 'lot': 11, 'practice': 12, 'spends': 13, 'time': 14, 'blood': 15, 'cause': 16, 'experts': 17, 'health': 18, 'increased': 19, 'may': 20, 'pressure': 21, 'suggest': 22, 'tension': 23, 'better': 24, 'drive': 25, 'feel': 26, 'never': 27, 'often': 28, 'perform': 29, 'school': 30, 'seems': 31, 'well': 32, 'professionals': 33, 'say': 34}

[ word_dictionary.id2token ]
{0: ',', 1: '.', 2: 'brocolli', 3: 'brother', 4: 'eat', 5: 'good', 6: 'likes', 7: 'mother', 8: 'around', 9: 'baseball', 10: 'driving', 11: 'lot', 12: 'practice', 13: 'spends', 14: 'time', 15: 'blood', 16: 'cause', 17: 'experts', 18: 'health', 19: 'increased', 20: 'may', 21: 'pressure', 22: 'suggest', 23: 'tension', 24: 'better', 25: 'drive', 26: 'feel', 27: 'never', 28: 'often', 29: 'perform', 30: 'school', 31: 'seems', 32: 'well', 33: 'professionals', 34: 'say'}


다음으로, Dictionary객체의 .doc2bow()를 사용해 document-term matrix를 만듭니다 :



In [90]:
dt_matrix = [word_dictionary.doc2bow(text) for text in texts]  # Tuple (Term ID, Term Frequency)
print("doc_set[0]:", doc_set[0])
print(dt_matrix[0])
print(dt_matrix)
print("::: corpus", ":" * 100, "\n")


doc_set[0]: Brocolli is good to eat. My brother likes to eat good brocolli, but not my mother.
[(0, 1), (1, 2), (2, 2), (3, 1), (4, 2), (5, 2), (6, 1), (7, 1)]
[[(0, 1), (1, 2), (2, 2), (3, 1), (4, 2), (5, 2), (6, 1), (7, 1)], [(1, 1), (3, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1)], [(1, 1), (10, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1)], [(0, 1), (1, 1), (3, 1), (7, 1), (21, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1)], [(1, 1), (2, 1), (5, 1), (18, 2), (33, 1), (34, 1)]]
::: corpus :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::: 



마지막으로, LDA model을 생성해 토픽 모델링을 수행하고, 결과를 확인합니다. 다만 수행시에 topic으로 생성될 집합의 숫자를 num_topics에 지정해줘야 합니다 :



In [91]:
ldamodel = gensim.models.ldamodel.LdaModel(dt_matrix, num_topics=3, id2word=word_dictionary.id2token, passes=20)
print("\n[ ldamodel ]")
print(ldamodel)
print("\n[ ldamodel.print_topics ]")
print(ldamodel.print_topics(num_topics=3, num_words=3))


[ ldamodel ]
LdaModel(num_terms=35, num_topics=3, decay=0.5, chunksize=2000)

[ ldamodel.print_topics ]
[(0, '0.104*"." + 0.080*"brocolli" + 0.080*"good"'), (1, '0.066*"brother" + 0.066*"mother" + 0.065*"."'), (2, '0.029*"." + 0.029*"pressure" + 0.029*"driving"')]


corpus의 길이가 짧아 주제를 파악하긴 어렵지만, 0번 토픽은 브로콜리, 1번 토픽은 가족, 2번 토픽은 운전이라는 주제가 될 수 있을 것 같습니다.

num_topics : 몇개의 주제로 나눌것인가 

id2word : 전체 토큰이 저장된 dictionary

passes : 정확도 (숫자가 클수록 정확하지만 속도가 느려질수도 있다.)